In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
# Training tweets as arrays
with open("train_0_100000.obj", "rb") as tr1:
    train_0 = pickle.load(tr1)
with open("train_100000_200000.obj", "rb") as tr2:
    train_1 = pickle.load(tr2)
with open("train_200000_300000.obj", "rb") as tr3:
    train_2 = pickle.load(tr3)
with open("train_300000_400000.obj", "rb") as tr4:
    train_3 = pickle.load(tr4)
with open("train_400000_end.obj", "rb") as tr5:
    train_4 = pickle.load(tr5)
# training labels
with open("train/data/tweet_by_ID_02_12_2019__10_46_39.txt.labels", "r") as tr_labels:
    train_labels = tr_labels.readlines()
# Test tweets as arrays
with open("test.obj", "rb") as test_tweets:
    test_twts = pickle.load(test_tweets)
# test labels
with open("test/us_test.labels", "r") as tl:
    test_labels = tl.readlines()

In [3]:
train_whole = np.array([])
train_whole = np.append(train_whole, train_0)
train_whole = np.append(train_whole, train_1)
train_whole = np.append(train_whole, train_2)
train_whole = np.append(train_whole, train_3)
train_whole = np.append(train_whole, train_4)
train_whole = np.reshape(train_whole, (437161,1024))
print(train_whole.shape)

(437161, 1024)


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [10]:
def Eval_LSTM(X, Y, lstm):
    num_correct = 0
    for i in range(len(X)):
        logProbs = lstm.forward(X[i], train=False)
        pred = torch.argmax(logProbs)
        if pred == int(Y[i]):
            num_correct += 1
    print("Accuracy: %s" % (float(num_correct) / float(len(X))))

In [19]:
class LSTM(nn.Module):
    def __init__(self, X, Y,NUM_CLASSES=20):
        super(LSTM, self).__init__()
        self.lstm = nn.LSTM(1024,20)
        self.hidden = (torch.randn(1, 1, 20),
                  torch.randn(1, 1, 20))

    def forward(self, X, train=False):
        #TODO: Implement forward computation
        X = X.view(1,1,-1).float()
        #X = torch.DoubleTensor(X)
        #print(X)
        out, self.hidden = self.lstm(X, self.hidden)
        return out

In [22]:
def Train_LSTM(X, Y,n_iter):
    print("Start Training!")
    lstm = LSTM(X, Y)
    #TODO: initialize optimizer.
    #optimizer = optim.SGD(mlp.parameters(), lr=.01)
    optimizer = optim.Adam(lstm.parameters(), lr=1e-5)
    Loss = torch.nn.CrossEntropyLoss()

    for epoch in range(n_iter):
        total_loss = 0.0
        for i in range(len(X)):
            #TODO: compute gradients, do parameter update, compute loss.
            lstm.zero_grad()
            probs = lstm.forward(X[i])
            probs = probs.squeeze(0)
            y = float(Y[i])
            loss = Loss(probs,torch.Tensor([y]).long())
            total_loss += loss
            loss.backward(retain_graph=True)
            lstm.hidden[0].detach_()
            lstm.hidden[1].detach_()
            optimizer.step()
            
        print(f"loss on epoch {epoch} = {total_loss}")
    return lstm

In [ ]:
train_tensor = torch.from_numpy(train_whole)
lstm = Train_LSTM(train_tensor, train_labels,25)

Start Training!
loss on epoch 0 = 1159363.375
loss on epoch 1 = 1121209.25


In [11]:
X_test = torch.from_numpy(test_twts)
Eval_LSTM(X_test, test_labels, lstm)

Accuracy: 0.2833
